In [ ]:
# Imports
import sys
sys.path.append("Program")

import datetime as dt
from fundamentals import *
from helper_functions import get_current_date, get_df, generate_end_dates, merge_stocks, stock_market
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from plot import *
from matplotlib.ticker import MaxNLocator
from scipy.stats import linregress
from stock_screener import create_rs_volume_df, stoploss_target
from technicals import *

In [ ]:
# Start of the program
start = dt.datetime.now()

# Variables
NASDAQ_all = True
period_hk = 60 # Period for HK stocks
period_us = 252 # Period for US stocks
RS = 90
factors = [1, 1, 1]
backtest = False

# Index
index_name = "^GSPC"
index_dict = {"^HSI": "HKEX", "^GSPC": "S&P 500", "^IXIC": "NASDAQ Composite"}

# Get the infix
infix = get_infix(index_name, index_dict, NASDAQ_all)

# Get the current date
current_date = get_current_date(start, index_name)

# Define the result folder
result_folder = "Result"

In [ ]:
end_date = current_date

# Check if there are pre-existing data
current_files = [file for file in os.listdir(result_folder) if file.startswith(f"{infix}rsvolume_")]

# Get the list of dates
dates = [file.split("_")[-1].replace(".csv", "") for file in current_files]

In [ ]:
# Choose the stocks
stocks = ["POWL"]
for stock in stocks:
    df = get_df(stock, current_date)
    plot_close(stock, df)
    plot_volatility(stock, df, 252)
    plot_MFI_RSI(stock, df, 252)
    if stock.endswith(".HK"):
        plot_stocks(["^HSI", stock], current_date)
    else:
        plot_stocks(["^GSPC", stock], current_date)

In [ ]:
# Get the stop loss and target price of a stock
stock = "CORZ"
df = get_df(stock, current_date)
current_close = df["Close"].iloc[-1]
stoploss, stoploss_pct, target, target_pct = stoploss_target(stock, 12.5, "2024-10-07")
print(f"Plan for {stock}.")
print(f"Current close: {round(current_close, 2)}.")
print(f"SL {stoploss} ({stoploss_pct}%) TP {target} ({target_pct}%)")

In [ ]:
# Get the stock lists
hkex_df = pd.read_csv("Program/constituents-hsi.csv")
hk_stocks = hkex_df["Symbol"].tolist()
us_stocks = stock_market(current_date, current_date, index_name, NASDAQ_all)

# Get the price data of HSI
index_df = get_df(index_name, current_date)

In [ ]:
# Combine the long term and short term RS dataframes
def long_short_RS(stocks, index_df, index_name, index_dict, current_date, end_dates1, end_dates2, periods1, periods2, volume_filter=None):
    # Convert inputs to lists
    if not isinstance(end_dates1, list):
        end_dates1 = [end_dates1]
    if not isinstance(end_dates2, list):
        end_dates2 = [end_dates2]
    if not isinstance(periods1, list):
        periods1 = [periods1]
    if not isinstance(periods2, list):
        periods2 = [periods2]
        
    # Initialize an empty list to store the merged dataframes
    merged_dfs = []

    # Initialize an empty list to store the index returns
    index_returns = []

    # Iterate over all combinations
    for end_date1, end_date2, period1, period2 in zip(end_dates1, end_dates2, periods1, periods2):
        # Filter the data
        index_df1 = index_df[index_df.index <= end_date1]
        index_df2 = index_df[index_df.index <= end_date2]

        # Calculate the percent change of the index
        index_df1.loc[:, "Percent Change"] = index_df1["Close"].pct_change()
        index_df2.loc[:, "Percent Change"] = index_df2["Close"].pct_change()
        
        # Calculate the total return of the index
        index_return1 = (index_df1["Percent Change"] + 1).tail(period1).cumprod().iloc[-1]
        index_return2 = (index_df2["Percent Change"] + 1).tail(period2).cumprod().iloc[-1]
        index_shortName = index_dict[f"{index_name}"]
        print(f"Return for {index_shortName} between {index_df1.index[-period1].strftime('%Y-%m-%d')} and {end_date1}: {index_return1:.2f}")
        print(f"Return for {index_shortName} between {index_df2.index[-period2].strftime('%Y-%m-%d')} and {end_date2}: {index_return2:.2f}")

        index_returns.extend([index_return1, index_return2])
        
    rs_dfs, volume_dfs, _ = create_rs_volume_df(stocks, current_date, end_dates1 + end_dates2, periods1 + periods2, index_returns, index_shortName, result_folder, infix, True, print_multiple=False)

    # Separate the dataframes into two halves
    length_df = len(rs_dfs) // 2
    rs_dfs1, rs_dfs2 = rs_dfs[:length_df], rs_dfs[length_df:]
    volume_dfs1, volume_dfs2 = volume_dfs[:length_df], volume_dfs[length_df:]

    for rs_df1, rs_df2, volume_df1, volume_df2 in zip(rs_dfs1, rs_dfs2, volume_dfs1, volume_dfs2):
        if volume_filter is not None:
            volume_df1 = volume_df1[(volume_df1["Volume SMA 5 Rank"] <= volume_filter) | (volume_df1["Volume SMA 20 Rank"] <= volume_filter)]
            volume_df2 = volume_df2[(volume_df2["Volume SMA 5 Rank"] <= volume_filter) | (volume_df2["Volume SMA 20 Rank"] <= volume_filter)]

            # Filter rs_df1 and rs_df2 based on the tickers present in volume dataframes
            rs_df1 = rs_df1[rs_df1["Ticker"].isin(set(volume_df1["Ticker"]))]
            rs_df2 = rs_df2[rs_df2["Ticker"].isin(set(volume_df2["Ticker"]))]

        # Merge and clean data
        merged_df = pd.merge(rs_df1, rs_df2, on="Ticker", suffixes=(" 1", " 2"))
        merged_df = merged_df.rename(columns={"RS 1": "Long-term RS", "RS 2": "Short-term RS"}).dropna()
        merged_dfs.append(merged_df)

    return merged_dfs[0] if len(merged_dfs) == 1 else merged_dfs

In [ ]:
# Variables
end_date1 = "2024-09-12"
end_date2 = "2024-10-07"
period1 = 252
period2 = 20

stocks = hk_stocks

# Get the merged dataframe
merged_df = long_short_RS(stocks, index_df, index_name, index_dict, current_date, end_date1, end_date2, period1, period2)

In [ ]:
stock_star = "9618.HK"

# Scatter plot of short-term RS against long-term RS
plt.figure(figsize=(10, 6))
plt.scatter(merged_df["Long-term RS"], merged_df["Short-term RS"], color="blue", marker="x")

# Highlight the specific ticker with a star
star = merged_df[merged_df["Ticker"] == stock_star]
if not star.empty:
    plt.scatter(star["Long-term RS"], star["Short-term RS"], color="gold", edgecolor="black", marker="*", s=100, label=stock_star)

# Plot a red vertical line at long-term RS = 20
plt.axvline(x=20, color="red", linestyle="--")

# Calculate the slope and R^2
slope, intercept, r_value, p_value, std_err = linregress(merged_df["Long-term RS"], merged_df["Short-term RS"])
r_squared = r_value**2

# Create the regression line
x_values = np.linspace(0, 100, 100)
y_values = slope * x_values + intercept
plt.plot(x_values, y_values, color="black", linestyle="--", label=fr"slope$={slope:.2f}$, $R^2={r_squared:.2f}$")

# Set the axes labels
plt.xlabel("Long-term RS")
plt.ylabel("Short-term RS")

# Set the title
plt.title(f"Long-term vs Short-term RS ({end_date1} to {end_date2})")

# Set the legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Define the end dates
end_dates = generate_end_dates(3, current_date, interval="1w")

In [ ]:
# Initialize two empty lists to store the RS slopes and R^2 values
rs_slopes = []
r_squareds = []

# Define the end dates and periods
end_dates1 = []
end_dates2 = []
for i in range(len(end_dates) - 1):
    end_date = end_dates[i]
    end_dates1.append(end_date)
    end_dates2.append((dt.datetime.strptime(end_date, "%Y-%m-%d") + relativedelta(days=20)).strftime("%Y-%m-%d"))
periods1 = [252] * len(end_dates1)
periods2 = [20] * len(end_dates2)

# Get the merged dataframe
merged_dfs = long_short_RS(stocks, index_df, index_name, index_dict, current_date, end_dates1, end_dates2, periods1, periods2)

# Iterate over merged dataframe
for merged_df in merged_dfs:
    # Calculate the slope and R^2
    rs_slope, _, r_value, _, _ = linregress(merged_df["Long-term RS"], merged_df["Short-term RS"])
    r_squared = r_value**2
    rs_slopes.append(rs_slope)
    r_squareds.append(r_squared)

    # # Scatter plot of short-term RS against long-term RS
    # plt.figure(figsize=(10, 6))
    # plt.scatter(merged_df["Long-term RS"], merged_df["Short-term RS"], color="blue", marker="x")

    # # Highlight the specific ticker with a star
    # star = merged_df[merged_df["Ticker"] == "9618.HK"]
    # if not star.empty:
    #     plt.scatter(star["Long-term RS"], star["Short-term RS"], color="gold", edgecolor="black", marker="*", s=100, label="9618.HK")

    # # Plot a red vertical line at long-term RS = 20
    # plt.axvline(x=20, color="red", linestyle="--")
    
    # # Create the regression line
    # x_values = np.linspace(0, 100, 100)
    # y_values = slope * x_values + intercept
    # plt.plot(x_values, y_values, color="black", linestyle="--", label=fr"slope$={slope:.2f}$, $R^2={r_squared:.2f}$")

    # # Set the axes labels
    # plt.xlabel("Long-term RS")
    # plt.ylabel("Short-term RS")

    # # Set the title
    # plt.title(f"Long-term vs short-term RS ({start_date} to {end_date})")

    # # Set the legend
    # plt.legend()

    # # Show the plot
    # plt.show()

In [ ]:
# Filter the dataframe
index_df = index_df.loc[(index_df.index >= end_dates[0]) & (index_df.index <= end_dates[-1])]

In [ ]:
# Create a figure with four subplots, one for the closing price, one for the RS slope, one for the R^2 values, and one for the Z-Score of product
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(8, 8), gridspec_kw={"height_ratios": [3, 0.5, 0.5, 1]}, sharex=True)

# Plot the closing price on the first subplot
ax1.plot(index_df["Close"], label="Close")

# Set the y label of the first subplot
ax1.set_ylabel("Price")

# Set the x limit of the first subplot
ax1.set_xlim(index_df.index[0], index_df.index[-1])

# Create a dataframe for RS slope
rs_slope_df = pd.DataFrame({"RS Slope": rs_slopes}, index=pd.to_datetime(end_dates2))

# Plot the RS slope on the second subplot
ax2.plot(rs_slope_df["RS Slope"], color="orange")

# Add a horizontal line at y=0
ax2.axhline(y=0, color="black", linestyle="--", linewidth=0.5)

# Set the y label for the second subplot
ax2.set_ylabel("RS slope")

# Create a dataframe for R^2
r_squareds_df = pd.DataFrame({"R^2": r_squareds}, index=pd.to_datetime(end_dates2))

# Plot the R^2 on the third subplot
ax3.plot(r_squareds_df, color="orange")

# Set the y label for the third subplot
ax3.set_ylabel(r"$R^2$")

# Calculate the z-scores of the product of RS slope and R^2
rs_slopes_r2 = np.array(rs_slopes) * r_squareds
rs_slopes_r2_mean = np.mean(rs_slopes_r2)
rs_slopes_r2_std = np.std(rs_slopes_r2)
z_scores = (rs_slopes_r2 - rs_slopes_r2_mean) / rs_slopes_r2_std

# Create a dataframe for z-scores
z_scores_df = pd.DataFrame({"Z-Score": z_scores}, index=pd.to_datetime(end_dates2))

# Plot the z-scores of the product of RS slope and R^2 on the fourth subplot
ax4.plot(z_scores_df, color="orange")

# Add a horizontal line at y=0
ax4.axhline(y=0, color="black", linestyle="--", linewidth=0.5)

# Add a red dotted line at y=2
ax4.axhline(y=2, color="red", linestyle="dotted")

# Add a red dotted line at y=-2
ax4.axhline(y=-2, color="red", linestyle="dotted")

# Set the y label for the fourth subplot
ax4.set_ylabel("Combined Z-Score")

# Ensure y-axis ticks are integers
ax4.yaxis.set_major_locator(MaxNLocator(integer=True))

# Set the x label
plt.xlabel("Date")

# Set the title
plt.suptitle(f"RS slope for {index_name}")

# Set the legend
ax1.legend()

# Adjust the spacing between subplots
plt.tight_layout()

# Save the plot
plt.savefig(f"Result/Figure/RSslope{index_name}.png", dpi=300)

# Show the plot
plt.show()

In [ ]:
# Variables
end_date1 = "2024-10-28"
end_date2 = "2024-11-17"
period1 = 252
period2 = 20

stocks = hk_stocks

# Get the merged dataframe
merged_df = long_short_RS(stocks, index_df, index_name, index_dict, current_date, end_date1, end_date2, period1, period2)

In [ ]:
stock_star = "9618.HK"

# Scatter plot of short-term RS against long-term RS
plt.figure(figsize=(10, 6))
plt.scatter(merged_df["Long-term RS"], merged_df["Short-term RS"], color="blue", marker="x")

# Highlight the specific ticker with a star
star = merged_df[merged_df["Ticker"] == stock_star]
if not star.empty:
    plt.scatter(star["Long-term RS"], star["Short-term RS"], color="gold", edgecolor="black", marker="*", s=100, label=stock_star)

# Plot a red vertical line at long-term RS = 20
plt.axvline(x=20, color="red", linestyle="--")

# Calculate the slope and R^2
slope, intercept, r_value, p_value, std_err = linregress(merged_df["Long-term RS"], merged_df["Short-term RS"])
r_squared = r_value**2

# Create the regression line
x_values = np.linspace(0, 100, 100)
y_values = slope * x_values + intercept
plt.plot(x_values, y_values, color="black", linestyle="--", label=fr"slope$={slope:.2f}$, $R^2={r_squared:.2f}$")

# Set the axes labels
plt.xlabel("Long-term RS")
plt.ylabel("Short-term RS")

# Set the title
plt.title(f"Long-term vs Short-term RS ({end_date1} to {end_date2})")

# Set the legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Calculate the new stop loss
price_avg = 13.461
stoploss_2r = price_avg * (1 - stoploss_pct / 100)
stoploss_2r = round(stoploss_2r, 2)
stoploss_1r = price_avg * (1 - stoploss_pct / 100 / 2)
stoploss_1r = round(stoploss_1r, 2)
stoploss_0r = price_avg
stoploss_0r = round(stoploss_0r, 2)
print(f"SL {stoploss_2r} ({stoploss_pct}%)")
print(f"SL {stoploss_1r} ({stoploss_pct / 2}%)")
print(f"SL {stoploss_0r} (0.0%)")

In [ ]:
show = 252 * 3
stocks = ["GC=F", "SI=F", "HG=F"]
metal_df = merge_stocks(stocks, current_date)
metal_df["Gold/Silver Ratio"] = metal_df["Close (GC=F)"] / metal_df["Close (SI=F)"]
metal_df["Gold/Copper Ratio"] = metal_df["Close (GC=F)"] / metal_df["Close (HG=F)"]
metal_df = calculate_ZScore(metal_df, ["Gold/Silver Ratio", "Gold/Copper Ratio"], 252)

# Restrict the dataframe
metal_df = metal_df[- show:]

# Create a figure with three subplots, one for the metal prices, one for the ratios, one for the ratios z-score
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 8), gridspec_kw={"height_ratios": [3, 1, 1]}, sharex=True)

# Plot the metal prices on the first subplot
close_goldfirst = metal_df["Close (GC=F)"].iloc[0]
close_silverfirst = metal_df["Close (SI=F)"].iloc[0]
close_copperfirst = metal_df["Close (HG=F)"].iloc[0]
ax1.plot(100 / close_goldfirst * metal_df["Close (GC=F)"], label="Gold (scaled)", color="gold")
ax1.plot(100 / close_silverfirst * metal_df["Close (SI=F)"], label="Silver (scaled)", color="silver")
ax1.plot(100 / close_copperfirst * metal_df["Close (HG=F)"], label="Copper (scaled)", color="peru")

# Set the label of the first subplot
ax1.set_ylabel("Price")

# Set the x limit of the first subplot
ax1.set_xlim(metal_df.index[0], metal_df.index[-1])

# Plot the ratios on the second subplot
goldsilver_ratio_first = metal_df["Gold/Silver Ratio"].iloc[0]
goldcopper_ratio_first = metal_df["Gold/Copper Ratio"].iloc[0]
ax2.plot(100 / goldsilver_ratio_first * metal_df["Gold/Silver Ratio"], color="silver")
ax2.plot(100 / goldcopper_ratio_first * metal_df["Gold/Copper Ratio"], color="peru")

# Set the y label of the second subplot
ax2.set_ylabel("Ratio wrt Gold")

# Plot the ratios z-score on the third subplot
ax3.plot(metal_df["Gold/Silver Ratio Z-Score"], color="silver")
ax3.plot(metal_df["Gold/Copper Ratio Z-Score"], color="peru")
ax3.axhline(y=2, linestyle="dotted", label="Undervalued", color="green")
ax3.axhline(y=-2, linestyle="dotted", label="Overvalued", color="red")

# Set the y label of the third subplot
ax3.set_ylabel("Ratio z-score")

# Set the x label
plt.xlabel("Date")

# Set the title
plt.suptitle(f"Metal prices comparison")

# Combine the legends and place them at the top subplot
handles, labels = ax1.get_legend_handles_labels()
handles += ax3.get_legend_handles_labels()[0]
labels += ax3.get_legend_handles_labels()[1]
ax1.legend(handles, labels)

# Adjust the spacing between subplots
plt.tight_layout()

# Save the plot
plt.savefig("Result/Figure/metalcompare.png", dpi=300)    

# Show the plot
plt.show()

In [ ]:
# Choose the stock
stock = "^HSI"

# Get the price data of the stock
df = get_df(stock, current_date)

# Add indicators
df = add_indicator(df)
df = calculate_ZScore(df, ["MFI", "RSI"], period=252*15)

# Save the data of the index to a .csv file
filename = f"Price data/{stock}_{current_date}.csv"
df.to_csv(filename)

periods = [5, 10, 15, 20, 30, 60]
for period in periods:
    df[f"Close {period} Later"] = df["Close"].shift(- period)
    df[f"{period} Days Return (%)"] = ((df[f"Close {period} Later"] / df["Close"]) - 1) * 100

# Filter for MFI/RSI Z-Score >= 2.5
df_MFIRSI_filter = df[(df["MFI Z-Score"] >= 2.5)]
print(df_MFIRSI_filter)

# Plot histogram
for period in periods:
    # Create a figure
    plt.figure(figsize=(10, 6))

    # Plot the histogram
    plt.hist(df_MFIRSI_filter[f"{period} Days Return (%)"].dropna(), bins=30, label=f'{period} Days Return (%)')

    # Calculate the mean
    mean = df_MFIRSI_filter.loc[:, f"{period} Days Return (%)"].mean()

    # Draw a vertical line at the mean
    plt.axvline(mean, color="red", linestyle="dashed", linewidth=1.5, label=f"Mean: {mean:.2f}%")

    # Set the y-axis ticks to integers
    y_ticks = np.arange(0, plt.ylim()[1] + 1, 1)
    plt.yticks(y_ticks)

    # Set the labels
    plt.xlabel("Return (%)")
    plt.ylabel("Count")

    # Set the title
    plt.title(rf"{period} days return when MFI Z-Score$\geq 2.5$ (%)")

    # Set the legend
    plt.legend()

    # Adjust the spacing
    plt.tight_layout()

    # Save the plot
    plt.savefig(f"Result/Figure/{period}returnMFIZgeq2.5.png", dpi=300)

    # Show the plot
    plt.show()